# Ciclo ciarcadiano metodo 1

In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier    
def extract_circadian_cycle(data):
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/24)
    
    # Filtrar para mantener solo frecuencias cercanas a 1 ciclo diario
    fft_filtered = fft_result.copy()
    fft_filtered[(freq < 0.8) | (freq > 1.2)] = 0
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))

def extract_circadian_cycle2(data, sample_rate=24, z=1):
    
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Mantener solo los armónicos en el rango [-z, z]
    fft_filtered = np.zeros_like(fft_result, dtype=complex)
    for k in range(-z, z + 1):
        idx = np.argmin(np.abs(freq - k))  # Encuentra el índice más cercano a la frecuencia k
        fft_filtered[idx] = fft_result[idx]
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))

# Configuración de datos de entrada
# Cargar los datos
nombres_csv = {1,2,3,4,468,479,4003,4151,4160,4173}
for nombre_csv in nombres_csv:
    df= pd.read_csv(f'Datos_actividad/{nombre_csv}_act.csv')

    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()


    # Función para procesar una columna de actividad y calcular niveles de estrés
    def process_activity(df, activity_column):
        daily_cycles = {}
        
        # Extraer ciclo circadiano para cada día y almacenarlo
        for date, group in df.groupby(df.index.date):
            if len(group) == 24:  # Verificar 24 horas de datos
                daily_cycles[date] = extract_circadian_cycle(group[activity_column].values)

        # Calcular el ciclo circadiano promedio
        average_cycle = np.mean(list(daily_cycles.values()), axis=0)
        #print(average_cycle)

        # Calcular distancias entre el ciclo promedio y los ciclos diarios
        hourly_distances = []
        for date, cycle in daily_cycles.items():
            for hour in range(24):
                hourly_distances.append({
                    'Fecha': date,
                    'Hora': hour,
                    #'distancia': euclidean(average_cycle[hour]-cycle[hour]),
                    'distancia': np.absolute(average_cycle[hour]-cycle[hour])
                })

        distance_df = pd.DataFrame(hourly_distances)
        
        # Calcular umbrales para clasificar niveles de estrés
        min_distance = distance_df['distancia'].min()
        max_distance = distance_df['distancia'].max()
        threshold1 = min_distance + (max_distance - min_distance) / 2
        threshold2 = threshold1 + (max_distance - threshold1) / 2
        threshold3 = max_distance
        
        # Establecer umbrales de estrés usando desviación estándar
        # mean_dist = distance_df['distancia'].mean()
        # std_dist = distance_df['distancia'].std()
        # threshold1 = mean_dist + std_dist
        # threshold2 = mean_dist + 2 * std_dist
        

        # Clasificación de niveles de estrés basada en los umbrales
        def get_stress_level(distance):
            if distance <= threshold1:
                return 'normal'
            elif distance <= threshold2:
                return 'alerta'
            else:
                return 'peligro'

        # Aplicar la función de nivel de estrés a cada distancia
        distance_df['nivel_estres'] = distance_df['distancia'].apply(get_stress_level)
        
        return distance_df[['Fecha', 'Hora', 'nivel_estres']]

    # Procesar la columna de actividad y calcular niveles de estrés
    stress_levels = process_activity(df, 'nivel_actividad')

    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels['Fecha'] = pd.to_datetime(stress_levels['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels['Hora'] = stress_levels['Hora'].astype(int)


    # Fusionar los DataFrames en df_final
    df_final = df.merge(stress_levels, on=['Fecha', 'Hora'], how='left')

    # Definir la fecha límite como Timestamp
    fecha_limite = pd.Timestamp('2024-06-30')

    # Dividir el DataFrame en dos partes
    df_entrenamiento = df_final[df_final['Fecha'] <= fecha_limite].copy()  # DataFrame antes de la fecha límite
    df_prueba = df_final[df_final['Fecha'] > fecha_limite].copy()  # DataFrame desde la fecha límite en adelante

    # Guardar los DataFrames en archivos CSV
    df_entrenamiento.to_csv(f'Datos_entrenamiento/{nombre_csv}_e.csv', index=False) 
    df_prueba.to_csv(f'Datos_prueba/{nombre_csv}_p.csv', index=False)


# Prueba de ciclo circadiano 2

In [1]:
import pandas as pd 
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data, sample_rate=24, z=1):
    
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Mantener solo los armónicos en el rango [-z, z]
    fft_filtered = np.zeros_like(fft_result, dtype=complex)
    for k in range(-z, z + 1):
        idx = np.argmin(np.abs(freq - k))  # Encuentra el índice más cercano a la frecuencia k
        fft_filtered[idx] = fft_result[idx]
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))

# Función para procesar una columna de actividad y calcular niveles de estrés
def process_activity(df, activity_column):
    # Crear ventanas móviles de 36 horas con paso de 1 hora
    window_size = 36
    step_size = 1

    series_results = []
    for start_idx in range(0, len(df) - window_size + 1, step_size):
        window_data = df.iloc[start_idx:start_idx + window_size]
        if len(window_data) == window_size:
            # Subseries A y B de 24 horas con desfase de 12 horas
            subseries_A = window_data.iloc[:24]
            subseries_B = window_data.iloc[12:36]
            
            # Extraer ciclos circadianos
            cycle_A = extract_circadian_cycle(subseries_A[activity_column].values)
            cycle_B = extract_circadian_cycle(subseries_B[activity_column].values)
            # Sincronizar modelos considerando el desfase de 12 horas
            synced_A = {i: cycle_A[i] for i in range(12, 24)}  # Ciclo A desde la hora 12 a 23
            synced_B = {i+12: cycle_B[i] for i in range(0, 12)}  # Ciclo B desde la hora 0 a 12
            
            # print('Cycle a')
            # print(cycle_A)
            # print('Cycle b')
            # print(cycle_B)
            
            # Comparar ciclos sincronizados (clave a clave)
            differences = []
            for key in synced_A:
                
                if key in synced_B:  # Verificar que exista la clave en ambos ciclos
                    differences.append((synced_A[key] - synced_B[key]) ** 2)
                    #print(differences)
                else:
                    differences.append(np.abs(synced_A[key] - 0))
            
            # Calcular distancia euclidiana acumulada
            dist_euclidiana = np.sqrt(sum(differences))
            #print(dist_euclidiana)
            
            # Almacenar resultados
            series_results.append({
                'Fecha': window_data.index[12].date(),
                'Hora': window_data.index[12].hour,
                'distancia': dist_euclidiana
            })
            # print(series_results)
            # print(window_data.index[12].hour)
            # Plotting
            # plt.figure(figsize=(18, 6))
            # hours = range(36)
            # plt.plot(hours, (window_data[activity_column].values), label="Nivel de actividad (Original)", linestyle='-', alpha=0.8)
            # plt.plot(range(24), cycle_A, 'k--', label="Ciclo A")
            # plt.plot(range(12, 36), cycle_B, 'k--', label="Ciclo B")
            # plt.title("Serie de 36 horas con ciclo circadiano A y B de 24 Horas")
            # plt.xlabel("Horas")
            # plt.ylabel("Nivel de actividad")
            # plt.legend()
            # plt.grid(True)
            # plt.show()

    # Guardar resultados como DataFrame
    distance_df = pd.DataFrame(series_results)
    
    # Establecer umbrales de estrés usando desviación estándar
    mean_dist = distance_df['distancia'].mean()
    std_dist = distance_df['distancia'].std()
    threshold1 = mean_dist + std_dist
    threshold2 = mean_dist + 2 * std_dist
    
    # Clasificación de niveles de estrés basada en los umbrales
    def get_stress_level(distance):
        if distance <= threshold1:
            return 'normal'
        elif distance <= threshold2:
            return 'alerta'
        else:
            return 'peligro'

    # Aplicar la función de nivel de estrés a cada distancia
    distance_df['nivel_estres'] = distance_df['distancia'].apply(get_stress_level)
    
    return distance_df[['Fecha', 'Hora','distancia', 'nivel_estres']]
    
           
# Configuración de datos de entrada
nombres_csv ={4}# {1, 2, 3, 4, 468, 479, 4003, 4151, 4160, 4173}
for nombre_csv in nombres_csv:
    df = pd.read_csv(f'Datos_actividad/{nombre_csv}_act.csv')
    # fecha_limite = '2024-01-02'
    # df = data[data['Fecha'] <= fecha_limite].copy()
    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()
    
    # Procesar  columnas de actividad y calcular niveles de estrés
    stress_levels = process_activity(df, 'nivel_actividad')
    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels['Fecha'] = pd.to_datetime(stress_levels['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels['Hora'] = stress_levels['Hora'].astype(int)
    
    # Fusionar todos los DataFrames en df_final
    df_final = df.merge(stress_levels, on=['Fecha', 'Hora'], how='left')
    df_final = df_final.iloc[24:-24]
    # Definir la fecha límite como Timestamp
    fecha_limite = pd.Timestamp('2024-06-30')

    # Dividir el DataFrame en dos partes
    df_entrenamiento = df_final[df_final['Fecha'] <= fecha_limite].copy()  # DataFrame antes de la fecha límite
    df_prueba = df_final[df_final['Fecha'] > fecha_limite].copy()  # DataFrame desde la fecha límite en adelante

    # Guardar los DataFrames en archivos CSV
    df_entrenamiento.to_csv(f'Datos_entrenamiento2/{nombre_csv}_e.csv', index=False) 
    df_prueba.to_csv(f'Datos_prueba2/{nombre_csv}_p.csv', index=False)

    
    
